In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 10.2 MB/s eta 0:00:00


In [ ]:
import os
from wordcloud import WordCloud
from konlpy.tag import Okt
from collections import Counter

In [ ]:
형태소분석기 = Okt()

In [72]:
from konlpy.tag import Okt
from gensim.models import FastText
from gensim.utils import simple_preprocess
import numpy as np
from scipy.spatial.distance import cosine

# 형태소 분석기 초기화
형태소분석기 = Okt()

# 문장 리스트
문장들 = [
    '카페가 너무 많아',
    '파스타 맛집이 생겼으면 좋겠다',
    '광운대 주변에는 왜 맥도날드나 롯데리아 같은 패스트푸드점이 없는거야?',
    '피자집 망했으면'
]

# 긍정 형용사, 부정 형용사, 긍정 부사, 부정 부사를 필터링할 키워드
긍정_형용사_키워드 = ['좋', '사랑', '기대', '행복', '정말']
부정_형용사_키워드 = ['싫', '별로', '왜', '없', '너무', '망']
긍정_부사_키워드 = 긍정_형용사_키워드  # 부사의 긍정적 키워드
부정_부사_키워드 = 부정_형용사_키워드  # 부사의 부정적 키워드

# 의미 없는 단어 필터링을 위한 집합
의미없는_단어 = {'은', '이', '가', '에', '의', '를', '와', '과', '도', '로', '서', '에서'}

# 단어 토큰화
토큰화된_문서들 = [simple_preprocess(문장) for 문장 in 문장들]

# FastText 모델 훈련
모델 = FastText(토큰화된_문서들, vector_size=50, window=3, min_count=1)

# 각 문장에서 가장 중요한 명사 단어를 찾기 위한 함수
def 중요한_단어_추출(명사_리스트, 모델):
    # 모델에 있는 단어만 필터링
    명사_필터링_단어들 = [단어 for 단어 in 명사_리스트 if 단어 in 모델.wv.index_to_key]

    if not 명사_필터링_단어들:
        # 명사 리스트에서 첫 번째 단어를 반환 (빈 경우)
        return 명사_리스트[0] if 명사_리스트 else None

    # 각 명사 단어의 벡터를 평균내어 문장 벡터를 생성
    벡터_값들 = [모델.wv.get_vector(단어) for 단어 in 명사_필터링_단어들]
    평균_벡터 = np.mean(벡터_값들, axis=0)

    # 명사 단어의 벡터와 평균 벡터의 유사도를 계산
    유사도_점수 = {}
    for 단어 in 명사_필터링_단어들:
        단어_벡터 = 모델.wv.get_vector(단어)
        유사도_점수[단어] = 1 - cosine(단어_벡터, 평균_벡터)

    # 가장 높은 유사도 점수를 가진 단어를 선택
    if 유사도_점수:
        가장_중요한_단어 = max(유사도_점수, key=유사도_점수.get)
        return 가장_중요한_단어
    else:
        # 명사 리스트에서 첫 번째 단어를 반환 (빈 경우)
        return 명사_리스트[0] if 명사_리스트 else None

# 문장별로 분석 및 결과 출력
for i, 문장 in enumerate(문장들):
    # 필터링된 결과 저장할 리스트 초기화
    명사 = []
    동사 = []
    긍정_형용사 = []
    부정_형용사 = []
    긍정_부사 = []
    부정_부사 = []

    # 각 문장에 대해 분석
    분석결과 = 형태소분석기.pos(문장)

    # 형태소 분석 결과의 단어와 품사 조정
    임시_단어 = []
    for 단어, 품사 in 분석결과:
        if 단어 in 의미없는_단어:
            continue

        # `별로`를 부정 부사로 강제로 분류
        if 단어 == '별로':
            부정_부사.append((단어, '부정 부사'))
        else:
            # 단어가 음절 단위로 나뉘는 경우를 처리
            if 품사 in ['Noun', 'Verb', 'Adjective', 'Adverb']:
                임시_단어.append((단어, 품사))

    # 필요한 품사 필터링
    for 단어, 품사 in 임시_단어:
        if 품사 == 'Noun':  # 명사
            명사.append(단어)

        elif 품사 == 'Adjective':  # 형용사 (긍정/부정 형용사로 분류)
            if any(keyword in 단어 for keyword in 긍정_형용사_키워드):
                긍정_형용사.append((단어, '긍정 형용사'))
            elif any(keyword in 단어 for keyword in 부정_형용사_키워드):
                부정_형용사.append((단어, '부정 형용사'))

        elif 품사 == 'Adverb':  # 부사 (긍정/부정 부사로 분류)
            if any(keyword in 단어 for keyword in 긍정_부사_키워드):
                긍정_부사.append((단어, '긍정 부사'))
            elif any(keyword in 단어 for keyword in 부정_부사_키워드):
                부정_부사.append((단어, '부정 부사'))

        elif 품사 == 'Verb':  # 동사 (동사로 분류)
            동사.append((단어, 품사))

    # 긍정과 부정의 개수 세기
    긍정_형용사_개수 = len(긍정_형용사)
    부정_형용사_개수 = len(부정_형용사)
    긍정_부사_개수 = len(긍정_부사)
    부정_부사_개수 = len(부정_부사)

    # 각 문장에서 가장 중요한 명사 추출
    가장_중요한_단어 = 중요한_단어_추출(명사, 모델)

    # 결과 출력
    print(f"{i+1} 번째 시도")
    print("명사:", 명사)
    print("동사:", 동사)
    print("긍정 형용사:", 긍정_형용사)
    print("부정 형용사:", 부정_형용사)
    print("긍정 부사:", 긍정_부사)
    print("부정 부사:", 부정_부사)
    print(f"긍정 표현 개수: {긍정_형용사_개수 + 긍정_부사_개수}")
    print(f"부정 표현 개수: {부정_형용사_개수 + 부정_부사_개수}")
    print(f"가장 중요해 보이는 명사: {가장_중요한_단어}")
    print()  # 빈 줄 추가


1 번째 시도
명사: ['카페']
동사: []
긍정 형용사: []
부정 형용사: []
긍정 부사: []
부정 부사: [('너무', '부정 부사')]
긍정 표현 개수: 0
부정 표현 개수: 1
가장 중요해 보이는 명사: 카페

2 번째 시도
명사: ['파스타', '맛집']
동사: [('생겼으면', 'Verb')]
긍정 형용사: [('좋겠다', '긍정 형용사')]
부정 형용사: []
긍정 부사: []
부정 부사: []
긍정 표현 개수: 1
부정 표현 개수: 0
가장 중요해 보이는 명사: 파스타

3 번째 시도
명사: ['광운대', '주변', '왜', '맥도날드', '롯데리아', '패스트푸드', '점']
동사: []
긍정 형용사: []
부정 형용사: [('없는거야', '부정 형용사')]
긍정 부사: []
부정 부사: []
긍정 표현 개수: 0
부정 표현 개수: 1
가장 중요해 보이는 명사: 광운대

4 번째 시도
명사: ['피자', '집']
동사: []
긍정 형용사: []
부정 형용사: [('망했으면', '부정 형용사')]
긍정 부사: []
부정 부사: []
긍정 표현 개수: 0
부정 표현 개수: 1
가장 중요해 보이는 명사: 피자

